In [4]:
import json
import os
from glob import glob
from subprocess import call
import time
import nibabel
import numpy as np
from joblib import Parallel, delayed
from skimage.exposure import rescale_intensity
from scipy.ndimage.measurements import label
import matplotlib.pyplot as plt

/tmp/ipykernel_273653/3411625526.py:10: DeprecationWarning: Please use `label` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  from scipy.ndimage.measurements import label


### Data Preparation

In [3]:
import concurrent.futures

# Preparing the data for the model
def get_folder_paths(directory):
    folder_paths = [os.path.join(directory, folder) for folder in os.listdir(directory) if os.path.isdir(os.path.join(directory, folder))]
    return folder_paths


# Function to load a NIfTI file
def load_nifty(directory, example_id, suffix):
    return nibabel.load(os.path.join(directory, example_id + "_" + suffix + ".nii.gz"))

# Function to load all sequences for a given example
def load_channels(d, example_id):
    return [load_nifty(d, example_id, suffix) for suffix in ["flair", "t1", "t1ce", "t2"]]

# Function to get data from a NIfTI file
def get_data(nifty, dtype="int16"):
    if dtype == "int16":
        data = np.abs(nifty.get_fdata().astype(np.int16))
        data[data == -32768] = 0
        return data
    return nifty.get_fdata().astype(np.uint8)

# Function to prepare a NIfTI file
def prepare_nifty(d):
    example_id = d.split("/")[-1]
    flair, t1, t1ce, t2 = load_channels(d, example_id)
    affine, header = t1.affine, t1.header
    vol = np.stack([get_data(flair), get_data(t1), get_data(t1ce), get_data(t2)], axis=-1)
    vol = nibabel.nifti1.Nifti1Image(vol, affine, header=header)
    nibabel.save(vol, os.path.join(d, example_id + ".nii.gz"))

    if os.path.exists(os.path.join(d, example_id + "_seg.nii.gz")):
        seg = load_nifty(d, example_id, "seg")
        affine, header = seg.affine, seg.header
        vol = get_data(seg, "unit8")
        # changing label 4 with 3 (for Brats dataset)
        vol[vol == 4] = 3
        seg = nibabel.nifti1.Nifti1Image(vol, affine, header=header)
        nibabel.save(seg, os.path.join(d, example_id + "_seg.nii.gz"))

# making the model's required file structure
def prepare_dirs(data, train):
    img_path, lbl_path = os.path.join(data, "images"), os.path.join(data, "labels")
    call(f"mkdir {img_path}", shell=True)
    if train:
        call(f"mkdir {lbl_path}", shell=True)


    dirs = glob(os.path.join(data, "BraTS*"))    #hard code
    # dirs = glob(os.path.join(data, "ASH*"))
    for d in dirs:
        if "_" in d.split("/")[-1]:
            files = glob(os.path.join(d, "*.nii.gz"))
            for f in files:
                if "flair" in f or "t1" in f or "t1ce" in f or "t2" in f:
                    continue
                if "_seg" in f:
                    call(f"mv {f} {lbl_path}", shell=True)
                else:
                    call(f"mv {f} {img_path}", shell=True)
        call(f"rm -rf {d}", shell=True)


# making the model's required json file
def prepare_dataset_json(data, train):
    images, labels = glob(os.path.join(data, "images", "*")), glob(os.path.join(data, "labels", "*"))
    images = sorted([img.replace(data + "/", "") for img in images])
    labels = sorted([lbl.replace(data + "/", "") for lbl in labels])

    modality = {"0": "FLAIR", "1": "T1", "2": "T1CE", "3": "T2"}
    labels_dict = {"0": "background", "1": "edema", "2": "non-enhancing tumor", "3": "enhancing tumour"}
    if train:
        key = "training"
        print("images", images)
        print("labels", labels)
        data_pairs = [{"image": img, "label": lbl} for (img, lbl) in zip(images, labels)]
    else:
        key = "test"
        data_pairs = [{"image": img} for img in images]

    dataset = {
        "labels": labels_dict,
        "modality": modality,
        key: data_pairs,
    }

    with open(os.path.join(data, "dataset.json"), "w") as outfile:
        json.dump(dataset, outfile)

# Function to run a function in parallel
def run_parallel(func, args):
    return Parallel(n_jobs=os.cpu_count())(delayed(func)(arg) for arg in args)
        

# def run_parallel(func, args):
#     with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
#         results = list(executor.map(func, args))
#     return results        

# Main function to prepare the dataset
def prepare_dataset(data, train):
    print(f"Preparing BraTS21 dataset from: {data}")
    start = time.time()
    run_parallel(prepare_nifty, sorted(glob(os.path.join(data, "BraTS*"))))   #hard code
    # run_parallel(prepare_nifty, sorted(glob(os.path.join(data, "ASH*"))))
    prepare_dirs(data, train)
    prepare_dataset_json(data, train)
    end = time.time()
    print(f"Preparing time: {(end - start):.2f}")


In [4]:
# this path '/content/drive/MyDrive/data/BraTS2021_train' should contain the folder containig the data with the ground truth (T1,T2,T1ce,flair,seg)
prepare_dataset("/home/workstation04/GP_MMMAI/data/BraTS2021_train", True)
print("Finished!")

Preparing BraTS21 dataset from: /home/workstation04/GP_MMMAI/data/BraTS2021_train


images ['images/BraTS2021_01667.nii.gz', 'images/BraTS2021_01668.nii.gz', 'images/BraTS2021_01669.nii.gz', 'images/BraTS2021_01670.nii.gz', 'images/BraTS2021_01671.nii.gz', 'images/BraTS2021_01672.nii.gz', 'images/BraTS2021_01673.nii.gz', 'images/BraTS2021_01674.nii.gz', 'images/BraTS2021_01675.nii.gz', 'images/BraTS2021_01676.nii.gz', 'images/BraTS2021_01677.nii.gz', 'images/BraTS2021_01678.nii.gz', 'images/BraTS2021_01679.nii.gz', 'images/BraTS2021_01680.nii.gz', 'images/BraTS2021_01681.nii.gz', 'images/BraTS2021_01682.nii.gz', 'images/BraTS2021_01683.nii.gz', 'images/BraTS2021_01684.nii.gz', 'images/BraTS2021_01685.nii.gz', 'images/BraTS2021_01686.nii.gz', 'images/BraTS2021_01687.nii.gz', 'images/BraTS2021_01688.nii.gz', 'images/BraTS2021_01689.nii.gz', 'images/BraTS2021_01690.nii.gz', 'images/BraTS2021_01691.nii.gz', 'images/BraTS2021_01692.nii.gz', 'images/BraTS2021_01693.nii.gz', 'images/BraTS2021_01694.nii.gz', 'images/BraTS2021_01695.nii.gz', 'images/BraTS2021_01696.nii.gz', 'i

In [5]:
import json

# Specify the path to your JSON file
json_file_path = '/home/workstation04/GP_MMMAI/data/BraTS2021_train/dataset.json'

# Open the JSON file and load its contents
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)
len(data['training'])

99

### Data Preprocessing

In [7]:
!python3 /home/workstation04/GP_MMMAI/Notebooks/NVIDIA_nnUNet/DeepLearningExamples/PyTorch/Segmentation/nnUNet/preprocess.py -h

2024-02-03 18:34:05.301883: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-03 18:34:05.301925: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-03 18:34:05.301959: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-03 18:34:05.308388: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-03 18:34:06.016377: W tensorflow/compiler/

In [1]:
# Preprocessing the data (NVIDIA nnUNet)
# task 11 is set for brats
# change exec_mode to val --> wont expect seg
!python3 /home/workstation04/GP_MMMAI/Notebooks/NVIDIA_nnUNet/DeepLearningExamples/PyTorch/Segmentation/nnUNet/preprocess.py --task 11  --exec_mode training --data "/home/workstation04/GP_MMMAI/data" --results "/media/workstation04/New Volume/MMMAI/NVIDIA_PED" --ohe

print("Finished!")

2024-03-03 01:36:37.221638: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 01:36:37.221682: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 01:36:37.221718: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 01:36:37.228529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 01:36:38.200383: W tensorflow/compiler/

In [ ]:
# Preprocessing (3D UNet) / without one hot encoding (--ohe)
!python3 /home/workstation04/GP_MMMAI/Notebooks/NVIDIA_nnUNet/DeepLearningExamples/PyTorch/Segmentation/nnUNet/preprocess.py --task 11  --exec_mode training --data "/content/drive/MyDrive/data" --results "/content/drive/MyDrive/data"
print("Finished!")

In [3]:
import pickle
with open('/media/workstation04/New Volume/MMMAI/NVIDIA_PED/11_3d/config.pkl', 'rb') as f:
    data = pickle.load(f)
print(data)

{'patch_size': [128, 128, 128], 'spacings': [1.0, 1.0, 1.0], 'n_class': 4, 'in_channels': 5}


### Inference

in case of this error ModuleNotFoundError: No module named 'nnunet.nn_unet'
execute this command
``` bash
touch /home/workstation04/GP_MMMAI/Notebooks/NVIDIA_nnUNet/DeepLearningExamples/PyTorch/Segmentation/nnUNet/nnunet/__init__.py
```


### Validation set

In [2]:
# Prediction (NVIDIA nnUNet)
!python /home/workstation04/GP_MMMAI/Notebooks/NVIDIA_nnUNet/DeepLearningExamples/PyTorch/Segmentation/nnUNet/main.py --exec_mode "predict" --gpus 1 --depth 6 --filters 64 96 128 192 256 384 512 --min_fmap 2 --amp --save_preds --task 11 --data "/media/workstation04/New Volume/MMMAI/NVIDIA_PED/11_3d" --ckpt_path "/home/workstation04/GP_MMMAI/Notebooks/NVIDIA_nnUNet/nvidia_util/epoch=7-dice=84.40.ckpt" --results "/media/workstation04/New Volume/MMMAI/NVIDIA_PED/prediction" --tta


2024-03-03 01:48:35.115727: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 01:48:35.115771: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 01:48:35.115810: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 01:48:35.134078: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 01:48:36.189919: W tensorflow/compiler/

-------------------------------------------
### Test set

In [7]:
# pretrained on testset
!python /home/workstation04/GP_MMMAI/Notebooks/NVIDIA_nnUNet/DeepLearningExamples/PyTorch/Segmentation/nnUNet/main.py --exec_mode "predict" --gpus 1 --depth 6 --filters 64 96 128 192 256 384 512 --min_fmap 2 --amp --save_preds --task 11 --data "/media/workstation04/New Volume/MMMAI/NVIDIA_PED_Trial/PED_testset/11_3d" --ckpt_path "/media/workstation04/New Volume/MMMAI/NVIDIA_PED_Trial/Uncertainty_sampling180/checkpoints/epoch=14-dice=78.28.ckpt" --results "/media/workstation04/New Volume/MMMAI/NVIDIA_PED_Trial/PED_testset" --tta

2024-03-14 08:52:51.293282: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 08:52:51.293328: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 08:52:51.293370: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 08:52:51.300770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 08:52:52.353208: W tensorflow/compiler/

---------------------

In [8]:
# The prediction will produce a npy file containing the probability to belong to a class for each voxel
# so we need to set each voxel to the class with the highest probability
# and then save the segmentation as nifti file

# Function to set each voxel to the class with the highest probability
def to_lbl(pred):
    # pred.shape = (num_classes, depth , height, width)
    print(pred.shape)
    pred = np.argmax(pred, axis=0)
    # Transpose to be  (height, width, depth)
    pred = np.transpose(pred, (2, 1, 0)).astype(np.uint8)
    print(pred.shape)
    return pred

# Function to save the predicted segmentation as nifti file
def prepare_preditions(e):
    fname = e[0].split("/")[-1].split(".")[0]
    preds = [np.load(f) for f in e]
    p = to_lbl(np.mean(preds, 0))


    # img is the input volumes stacked together ,needed so that the saved prediction have the same affine (map voxel coordinates to real-world coordinates) and header (contains metadata information about the image)
    img = nibabel.load(f"/media/workstation04/New Volume/MMMAI/NVIDIA_PED_Trial/PED_testset/data/BraTS2021_train/images/{fname}.nii.gz")
    nibabel.save(
        nibabel.Nifti1Image(p, img.affine, header=img.header),
        os.path.join("/media/workstation04/New Volume/MMMAI/NVIDIA_PED_Trial/PED_testset/prediction_uncertainty180", fname + ".nii.gz"),
    )

In [9]:
preds = sorted(glob(f"/media/workstation04/New Volume/MMMAI/NVIDIA_PED_Trial/PED_testset/predictions_epoch=14-dice=78_28_task=11_fold=0_tta*"))
examples = list(zip(*[sorted(glob(f"{p}/*.npy")) for p in preds]))
print("Preparing final predictions")
for e in examples:
    prepare_preditions(e)
print("Finished!")

Preparing final predictions


(4, 155, 240, 240)
(240, 240, 155)
(4, 155, 240, 240)
(240, 240, 155)
(4, 155, 240, 240)
(240, 240, 155)
(4, 155, 240, 240)
(240, 240, 155)
(4, 155, 240, 240)
(240, 240, 155)
(4, 155, 240, 240)
(240, 240, 155)
(4, 155, 240, 240)
(240, 240, 155)
(4, 155, 240, 240)
(240, 240, 155)
(4, 155, 240, 240)
(240, 240, 155)
(4, 155, 240, 240)
(240, 240, 155)
Finished!
